In [ ]:
import re
import nltk
import spacy
import pandas as pd
from string import ascii_lowercase
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
from sklearn.feature_extraction.text import CountVectorizer

Read in the file previously created

In [ ]:
df = pd.read_pickle('C:/users/tyler/desktop/texts_lammetized.pkl')

In [ ]:
df

**Next Features:**  
1. proper nouns
2. convert emojies to text

### A Few Functions for Processing

In [ ]:
def simplify_elongated_words(df):  # replace 'aaaaaaaahhhhh' with 'aahh'
    counter = 0
    for letter in ascii_lowercase:
#         print('working on', letter)
        for row_idx, row in enumerate(df['tokenize_text']):
            for word_idx, word in enumerate(row):
                original_word = word
                while word != word.replace(letter*3, letter*2): # can strip out multiple letters?
                    word = word.replace(letter*3, letter*2) #strip out multiple letters!
                    df['tokenize_text'][row_idx][word_idx] = word
#                     print(df['tokenize_text'][row_idx][word_idx])
    return df

In [ ]:
def remove_stop_words(df):
    for row in df['tokenize_text']:
        for word in row:
            if word in stopwords.words('english'):
                row.remove(word)
    return df

In [ ]:
def custom_stop_words(df):
    # this takes a while
    custom_stop_list = ['-PRON-', 'pron', ' ', '-pron-']
#                         , 'know', 'like', 'good', 'time', 'want', 'll', 'liked'
#                         , 'yeah', 'ya', 'yes', 'think', 'just', 'thbe'
#                         , 'https', 'com', 'wh', 'ww', 'image', 'utm', 'fedex', 'feel'
#                         , 'ok', 'hey', 'know', 'let', 'want', 'wan', 'na', 'need', 'a'
#                         , 'say', 'look','love', 'oh', 'haha', 'cool', 'home', 'will', 'time']
    
    
    for idx, row in enumerate(df['tokenize_text']):
        new_row = row
        for word in row:
            if word in custom_stop_list:
                new_row = [x for x in new_row if x != word]
#                 print(idx, word, new_row)
                if idx%10000==0:
                    print('row', idx)
        df['tokenize_text'][idx] = new_row
               
    return df

In [ ]:
def list_into_string(df): # in order to use CountVectorizer which intakes strings, not lists
    for idx, row in enumerate(df['tokenize_text']):
        df['tokenize_text'][idx] = ' '.join(row)
    return df

In [ ]:
# adding some prints since this one can take a minute
# df = df.head(40)

print('simplifying words')
df = simplify_elongated_words(df)

print('removing stop words') # takes the longest
df = remove_stop_words(df)

print('removing custom stop words') # maybe don't do this?
df = custom_stop_words(df)

print('detokenizing...')
df = list_into_string(df)

In [ ]:
# cv = CountVectorizer(strip_accents='ascii', lowercase=False) #strip accents since i sometimes text in spanish
# X = cv.fit_transform(df['tokenize_text'])
# df2 = pd.DataFrame(X.toarray(), columns = cv.get_feature_names())

## Drop newly 'nan' or null rows

In [ ]:
blank_rows = df[df['tokenize_text']==''].index.tolist() # find blanks and put their index into a list
nan_rows = df[df['tokenize_text']=='nan'].index.tolist() # find 'nan's and put their index into a list

df.drop(blank_rows, inplace=True)
df.drop(nan_rows, inplace=True)

### Save

In [ ]:
df.to_pickle('C:/users/tyler/desktop/texts_docs.pkl') # save the cleaned version